# Exploring Evolutionary Algorithms w/ DEAP

In [49]:
# Utilities
from datetime import datetime
import pandas_datareader as pdr
import pandas as pd
import numpy as np
from typing import List, Tuple

symbols = pd.read_csv("data/sp500_symbols.csv")

symbols_df = pd.read_csv("data/sp500_symbols.csv")
symbols = symbols_df["Symbol"].tolist()

def convert_to_datetime(year: int, month: int, day: int) -> datetime:
    return datetime(year=year, month=month, day=day)

def get_ticker_data(ticker: str, start: datetime, end: datetime) -> pd.DataFrame:
    return pdr.get_data_yahoo(ticker, start=start, end=end)

def calculate_cagr(df: pd.DataFrame) -> float:
    days = (df.index[-1] - df.index[0]).days
    return ((((df['Adj Close'][-1]) / df['Adj Close'][1])) ** (365.0/days)) - 1

def calculate_portfolio_returns(weights: List[float], stock_symbols: List[str], start: datetime, end:datetime):
    dfs = [get_ticker_data(symbol, start, end) for symbol in stock_symbols]
    cagrs = [calculate_cagr(df) for df in dfs]    
    return np.dot(weights, cagrs)

def generate_random_dist(bins: int):
    return [round(val, 2) for val in np.random.dirichlet(np.ones(bins),size=1)[0].tolist()]

def generate_random_stocks(num_stocks: int) -> List[str]:
    return np.random.choice(symbols["Symbol"], size = num_stocks)

def generate_random_portfolio(num_stocks: int) -> Tuple[str, float]:
    random_stocks = generate_random_stocks(num_stocks)
    dist = generate_random_dist(num_stocks)
    return list(zip(random_stocks, dist))



In [60]:
class Stocks():

    def __init__(self, eval_start: datetime, eval_end: datetime, num_stocks: int):

        # Load stocks and grab n for the simulation
        symbols = pd.read_csv("data/sp500_symbols.csv")["Symbol"].tolist()
        stock_sample = np.random.choice(symbols, num_stocks).tolist()

        # convert stocks to ids
        stock2id = 
        

In [66]:
symbols = pd.read_csv("data/sp500_symbols.csv")["Symbol"].tolist()


['ALXN', 'NI', 'ICE', 'FLIR', 'CB', 'CMS', 'RL', 'RMD', 'BR', 'MET']